<a href="https://colab.research.google.com/github/maxrgnt/pythdc2-project2/blob/master/EDA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Panel Data
import pandas as pd
# System folders
import os
from pathlib import Path
# Visualization
import plotly.graph_objs as go
import seaborn as sns
%matplotlib inline

In [126]:
url = 'https://raw.githubusercontent.com/maxrgnt/pythdc2-project2/master/data/unemployment.csv'
bls = pd.read_csv(url)
print(bls.shape)
bls.sample(1)

(322, 5)


,State,Year,Abrv,UnemploymentRate,LaborRate
193,Texas,2009,TX,0.081924,0.658382


In [127]:
url = 'https://raw.githubusercontent.com/maxrgnt/pythdc2-project2/master/data/pctChangeGDP.csv'
bea = pd.read_csv(url)
print(bea.shape)
bea.sample(1)

(322, 4)


,State,Abrv,Year,Value
117,Michigan,MI,2004,2.0


In [128]:
# read in the data
url='https://raw.githubusercontent.com/maxrgnt/pythdc2-project2/master/data/borderCrossing.csv'
bts = pd.read_csv(url)
print(bts.shape)
bts.sample(1)

(114588, 8)


,Abrv,State,Longitude,Latitude,Border,Year,Measure,Value
269,WA,Washington,-122.44316,48.94802,US-Canada Border,2018,Bus Passengers,93


In [0]:
def checkPctChange(df):
  print(df[(df['Abrv']=='TX') & (df['Year']==2014)])
  print(df[(df['Abrv']=='TX') & (df['Year']==2015)])
  print(df[(df['Abrv']=='TX') & (df['Year']==2016)])

In [0]:
def calcPctChange(df, col, pctChange_id):
  for y in list(df['Year'].unique()):
    states = list(df['Abrv'].unique())
    for s in states:
      if y != df['Year'].min():
        t0 = df.loc[(df['Year'] == y-1) & (df['Abrv'] == s), col].tolist()[0]
        t1 = df.loc[(df['Year'] == y) & (df['Abrv'] == s), col].tolist()[0]
        df.loc[(df['Year']==y) & (df['Abrv']==s), pctChange_id+'_pctChange'] = ((t1/t0)-1)*100
      else:
        df.loc[(df['Year']==y) & (df['Abrv']==s), pctChange_id+'_pctChange'] = 'N/A'
  # set drop index for first year where pctChange not calculated
  dropIndex = df.loc[df[pctChange_id+'_pctChange']=='N/A'].index
  df.drop(dropIndex, inplace=True)

In [139]:
bts2 = bts.groupby(['Year','Abrv'])[['Value']].sum().reset_index()
bts2.sample(1)

,Year,Abrv,Value
233,2012,NM,1835195


In [140]:
calcPctChange(bts2, 'Value', 'border')
checkPctChange(bts2)

(322, 4)
(308, 4)
     Year Abrv     Value border_pctChange
263  2014   TX  80334766          4.71749
     Year Abrv     Value border_pctChange
277  2015   TX  82726100          2.97671
     Year Abrv     Value border_pctChange
291  2016   TX  85714503          3.61241


In [141]:
checkPctChange(bea)

     State Abrv  Year  Value
263  Texas   TX  2014    4.2
     State Abrv  Year  Value
277  Texas   TX  2015    0.2
     State Abrv  Year  Value
291  Texas   TX  2016   -0.2


In [142]:
calcPctChange(bls, 'UnemploymentRate', 'unemp')
checkPctChange(bls)

(322, 6)
(308, 6)
     State  Year Abrv  UnemploymentRate  LaborRate unemp_pctChange
263  Texas  2014   TX          0.053764    0.64867        -19.4988
     State  Year Abrv  UnemploymentRate  LaborRate unemp_pctChange
277  Texas  2015   TX          0.046521   0.637932        -13.4719
     State  Year Abrv  UnemploymentRate  LaborRate unemp_pctChange
291  Texas  2016   TX          0.048399   0.637718         4.03763
